### FAISS
Facebook AI Similarity search is a library for efficient similarity search and clustering of dense vector.It contains algorithms that search in set of vector of any size, upto ones that possibly donot fit in RAM. It also  contain suporting code for evaluation and parameter tunning.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [ ]:
embeddings=OllamaEmbeddings(model="gemma2:2b") 
db=FAISS.from_documents(docs,embeddings)
db

In [9]:
### querying
query = "how does the speaker describe the desired outcome of war?"
docs = db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [ ]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [12]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  8287.146),
 (Document(metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the

In [13]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.3731721043586731,
 0.27457094192504883,
 -0.7710204720497131,
 -1.097152829170227,
 -0.23380181193351746,
 0.004782151896506548,
 0.7259626388549805,
 0.3544575273990631,
 0.628194272518158,
 -0.21324406564235687,
 -0.8128089308738708,
 -1.2033085823059082,
 -1.1251107454299927,
 0.26568537950515747,
 -2.27152943611145,
 -1.5990058183670044,
 1.8067532777786255,
 -0.9284796714782715,
 -2.420724391937256,
 -0.8139026165008545,
 0.34738463163375854,
 -2.039062261581421,
 -0.8907489776611328,
 1.4101722240447998,
 -1.5311989784240723,
 -2.182032823562622,
 -0.001175203244201839,
 -0.8459579944610596,
 -0.9271553158760071,
 3.1291627883911133,
 -3.430896520614624,
 -1.282926082611084,
 0.7088844776153564,
 1.9401638507843018,
 1.945343017578125,
 1.6774144172668457,
 -0.893168568611145,
 1.4544475078582764,
 2.9443089962005615,
 -2.747864007949829,
 1.7508232593536377,
 1.094775915145874,
 -0.8801790475845337,
 -2.4953198432922363,
 -1.861513376235962,
 -2.367680549621582,
 0.0167887341

In [14]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those

In [15]:
### Saving And Loading
db.save_local("faiss_index")

In [16]:
### Loading
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [17]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those